In [1]:
from fastapi import FastAPI
from pydantic import BaseModel
from typing import Optional
from db import init_db, get_conn
from logic import choose_best_slot, set_slot

app = FastAPI(title="Warehouse MVP (no AI)")

class EventIn(BaseModel):
    direction: str   # IN / OUT / UNKNOWN
    ts_ms: int
    source: str = "esp32"

class AllocateReq(BaseModel):
    item_id: Optional[str] = None
    L: float
    W: float
    H: float
    est_weight: float = 0.0

class AllocateResp(BaseModel):
    slot_id: Optional[str]
    reason: str

@app.on_event("startup")
def startup():
    init_db()

@app.post("/event")
def post_event(e: EventIn):
    conn = get_conn()
    cur = conn.cursor()
    cur.execute(
        "INSERT INTO events(ts_ms, direction, source) VALUES (?,?,?)",
        (e.ts_ms, e.direction.upper(), e.source),
    )
    conn.commit()
    conn.close()
    return {"ok": True}

@app.get("/events")
def list_events(limit: int = 50):
    conn = get_conn()
    cur = conn.cursor()
    cur.execute("SELECT * FROM events ORDER BY id DESC LIMIT ?", (limit,))
    rows = [dict(r) for r in cur.fetchall()]
    conn.close()
    return rows

@app.post("/slots/seed")
def seed_slots():
    example = [
        ("A1", 120, 80, 150, 500),
        ("A2", 120, 80, 150, 500),
        ("B1", 120, 100, 200, 800),
    ]
    conn = get_conn()
    cur = conn.cursor()
    for slot_id, L, W, H, max_w in example:
        cur.execute("""
          INSERT OR IGNORE INTO slots(slot_id, L, W, H, max_weight, occupied)
          VALUES(?,?,?,?,?,0)
        """, (slot_id, L, W, H, max_w))
    conn.commit()
    conn.close()
    return {"ok": True, "seeded": len(example)}

@app.get("/slots")
def list_slots():
    conn = get_conn()
    cur = conn.cursor()
    cur.execute("SELECT * FROM slots ORDER BY slot_id ASC")
    rows = [dict(r) for r in cur.fetchall()]
    conn.close()
    return rows

@app.post("/allocate", response_model=AllocateResp)
def allocate(req: AllocateReq):
    slot = choose_best_slot(req.L, req.W, req.H, req.est_weight)
    if not slot:
        return AllocateResp(slot_id=None, reason="No eligible vacant slot (size/weight constraints).")
    return AllocateResp(slot_id=slot, reason="Best-fit vacant slot considering size + max_weight.")

@app.post("/slot/{slot_id}/occupy")
def occupy_slot(slot_id: str, item_id: str = "UNKNOWN", weight: float = 0.0):
    set_slot(slot_id, True, item_id, weight)
    return {"ok": True, "slot_id": slot_id, "occupied": True}

@app.post("/slot/{slot_id}/free")
def free_slot(slot_id: str):
    set_slot(slot_id, False, None, 0.0)
    return {"ok": True, "slot_id": slot_id, "occupied": False}


ModuleNotFoundError: No module named 'fastapi'